In [4]:
#hide
#default_exp showdoc
#default_cls_lvl 3

In [13]:
#export
from nbdev.imports import *
from nbdev.read import *
from nbdev.export import *
from nbdev.doclinks import *
from fastcore.utils import *

import pkg_resources,importlib

if IN_NOTEBOOK:
    from IPython.display import Markdown,display
    from IPython.core import page
else: Markdown,display,page = None,None,None

In [6]:
from fastcore.test import *

# Show doc

> Functions to show the doc cells in notebooks

In [16]:
#export
class NbdevLookup:
    def __init__(self, d, strip_all=None, strip_mods=None):
        settings = d['settings']
        if strip_mods is None: strip_mods = settings.get('strip_mods','').split()
        if strip_all  is None: strip_all  = settings.get('strip_all',settings.get('lib_name')).split()
        strip_mods = L(strip_mods)
        py_syms = merge(*d['syms'].values())
        for m in strip_all:
            _d = importlib.import_module(f"{m}._nbdev")._modidx.d
            stripped = {remove_prefix(k,f"{k_}."):v
                        for k_,v_ in _d['syms'].items() for k,v in v_.items()}
            py_syms = merge(stripped, py_syms)
        for m in strip_mods:
            stripped = {remove_prefix(k,f"{m}."):v for k,v in d['syms'][m].items()}
            py_syms = merge(stripped, py_syms)
        self.syms = merge(py_syms, d['mods'])

    def __call__(self, s):
        "URL for documentation for symbol `s`"
        return self.syms.get(s, None)

In [17]:
from nbdev import _modidx

In [18]:
c = NbdevLookup(_modidx.d, strip_all=[])
assert c('nbdev.read.cell_header').startswith('http')
assert not c('cell_header')
assert not c('DocLinks')

In [19]:
c = NbdevLookup(_modidx.d, strip_mods='nbdev.read', strip_all=[])
assert c('cell_header').startswith('http')
assert not c('DocLinks')

In [20]:
c = NbdevLookup(_modidx.d)
assert c('nbdev.read.cell_header').startswith('http')
assert c('cell_header').startswith('http')
assert c('DocLinks').startswith('http')

In [21]:
from nbdev._nbdev import *
assert nbdev_lookup('cell_header').startswith('http')
assert pkg_resources.parse_version(version).major>1

In [22]:
@functools.lru_cache
class DocLookup:
    def __init__(self):
        self.pkgs = L(o.load()() for o in pkg_resources.iter_entry_points(group='nbdev'))
    def __call__(self, k): return self.pkgs.map_first(Self.doc_link(k))

## Backticks

All the automatic documentation of functions and classes are generated with the `show_doc` function. It displays the name, arguments, docstring along with a link to the source code on GitHub.

In [73]:
md = """This is a link to `numpy.array` and to `read_nb` and to `L` but not a link to `foobar`.
And not a link to <code>dict2nb</code>.

    This is not a link to `dict2nb`

```
This isn't a link to `dict2nb` either
```"""

In [74]:
def _link_sym(m):
    l = m.group(1)
    s = DocLookup()(l)
    if s is None: return l
    return rf"[{l}]({s})"

In [75]:
_re_backticks = re.compile(r'`([^`\s]+)`')
def _link_line(l):
    return _re_backticks.sub(_link_sym, l)

In [76]:
in_fence=False
lines = md.splitlines()
for i,l in enumerate(lines):
    if l.startswith("```"): in_fence=not in_fence
    elif not l.startswith('    '): lines[i] = _link_line(l)

print('\n'.join(lines))

This is a link to [numpy.array](https://numpy.org/doc/reference/generated/numpy.array.html#numpy.array) and to read_nb and to L but not a link to foobar.
And not a link to <code>dict2nb</code>.

    This is not a link to `dict2nb`

```
This isn't a link to dict2nb either
```


## Export -

In [27]:
from nbdev.doclinks import nbdev_build_lib
nbdev_build_lib()